In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    value: str

In [2]:
NODE_A = "node_a"
NODE_B = "node_b"
NODE_C = "node_c"
NODE_D = "node_d"

In this case, we don't need to manually add edges in the StateGraph because the flow between the nodes is automatically controlled using the goto field inside the Command object returned by each node function. Each node explicitly mentions where the execution should jump next (goto=NODE_B, goto=NODE_C, etc.). This dynamic redirection acts as a virtual edge between nodes during runtime, so explicit add_edge statements are not required. This approach gives you full flexibility to decide the next node based on runtime conditions, user input (like in interrupt()), or any state logic, instead of fixing the graph structure at design time.

In [7]:
def node_a(state: State):
    print("Node A")
    return Command(
        goto=NODE_B,
        update={
            "value" : state["value"] + "a"
        }
    )

def node_b(state : State):
    print("NODE B")
    human_response = interrupt("Do you want to go to C or D? Type C/D")

    print("Human Review Values: ", human_response)

    if(human_response == "D"):
        return Command(
            goto=NODE_D,
            update={
                "value" : state["value"] + "b"
            }
        )
    else:
        return Command(
            goto=NODE_C,
            update={
                "value" : state["value"] + "b"
            }
        )
    
def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def node_d(state: State): 
    print("Node D")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)


graph = StateGraph(State)

graph.add_node(NODE_A, node_a)
graph.add_node(NODE_B, node_b)
graph.add_node(NODE_C, node_c)
graph.add_node(NODE_D, node_d)

graph.set_entry_point(NODE_A) 

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}
initialState = {
    "value": ""
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result


Node A
NODE B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:9e30f1b6-fd4b-c4c0-4fe0-f31b4b5f4a1e']),)}]

In [9]:
print(app.get_state(config).next)

('node_b',)


In [10]:
second_result = app.invoke(Command(resume="D"), config=config, stream_mode="updates")
second_result

NODE B
Human Review Values:  D
Node D


[{'node_b': {'value': 'ab'}}, {'node_d': {'value': 'abd'}}]